In [1]:
pip install pandas openpyxl plotly numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Парсинг данных из CSV
def parse_blade_data(file_path, x_row=0, z_row=2):
    df = pd.read_csv(file_path, header=None)
    if df.shape[0] <= z_row:
        raise ValueError(f"В файле меньше {z_row + 1} строк, невозможно извлечь Z")
    
    x_values = df.iloc[x_row, :].dropna().values
    z_values = df.iloc[z_row, :].dropna().values
    
    def is_valid_float(value):
        try:
            float(value)
            return True
        except (ValueError, TypeError):
            return False
    
    x_values = [x for x in x_values if is_valid_float(x) and str(x).strip() != '']
    z_values = [z for z in z_values if is_valid_float(z) and str(z).strip() != '']
    
    if len(x_values) != len(z_values):
        raise ValueError(f"Длины массивов X ({len(x_values)}) и Z ({len(z_values)}) не совпадают")
    
    try:
        x_values = np.array(x_values, dtype=float)
        z_values = np.array(z_values, dtype=float)
    except Exception as e:
        print("Ошибка преобразования в float:")
        print("X_values:", x_values)
        print("Z_values:", z_values)
        raise
    
    non_zero_mask = z_values != 0
    x_values = x_values[non_zero_mask]
    z_values = z_values[non_zero_mask]
    
    if len(x_values) == 0:
        raise ValueError("После исключения нулевых значений Z данные отсутствуют")
    
    return x_values, z_values

# Алгоритм определения начала и конца лопастей
def find_blade_boundaries(x_values, z_values, table_std_threshold=0.2, z_drop_threshold=3.0, min_table_points=4):
    """
    Parameters:
    - table_std_threshold: порог стандартного отклонения для определения поверхности стола
    - z_drop_threshold: порог резкого падения Z для конца лопасти (мм)
    - min_table_points: минимальное количество точек для определения поверхности стола
    """
    blades = []
    i = 0
    n = len(z_values)
    
    while i < n:
        # 1. Поиск поверхности стола
        table_end = i + min_table_points
        if table_end >= n:
            break
        
        window = z_values[i:table_end]
        if len(window) >= min_table_points and np.std(window) < table_std_threshold:
            # Найдена поверхность стола
            i = table_end
        else:
            i += 1
            continue
        
        # 2. Поиск начала лопасти (устойчивое уменьшение Z)
        blade_start = i
        while i < n and abs(z_values[i] - z_values[blade_start - 1]) < z_drop_threshold:
            i += 1
        if i == blade_start:
            continue  # Не найдено устойчивое уменьшение
        
        # 3. Поиск конца лопасти (резкое падение Z)
        blade_end = i
        while blade_end < n - 1:
            if abs(z_values[blade_end] - z_values[blade_end + 1]) >= z_drop_threshold:
                blade_end += 1
                break
            blade_end += 1
        
        if blade_end < n:
            blades.append((blade_start, blade_end))
        
        i = blade_end
    
    # Извлечение координат лопастей
    blade_x = []
    blade_z = []
    for start, end in blades:
        blade_x.extend(x_values[start:end])
        blade_z.extend(z_values[start:end])
    
    return np.array(blade_x), np.array(blade_z), blades

# Чтение данных
file_path = '/kaggle/input/lopatki2/3_4_brak_u_mer_sechenie_verh_-8.csv'
try:
    x_values, z_values = parse_blade_data(file_path)
except Exception as e:
    print(f"Ошибка при парсинге: {e}")
    df = pd.read_csv(file_path, header=None)
    print("Первые 5 строк CSV:")
    print(df.head())
    print("Значения X (первая строка):", df.iloc[0, :].values)
    print("Значения Z (третья строка):", df.iloc[2, :].values if df.shape[0] > 2 else "Меньше 3 строк")
    raise

# Поиск границ лопастей
try:
    blade_x, blade_z, blade_boundaries = find_blade_boundaries(x_values, z_values)
except Exception as e:
    print(f"Ошибка при определении границ лопастей: {e}")
    raise

# Вывод информации о найденных лопастях
print(f"Найдено лопастей: {len(blade_boundaries)}")
for i, (start, end) in enumerate(blade_boundaries):
    print(f"Лопасть {i+1}: X[{start}:{end}] = [{x_values[start]:.2f}, {x_values[end-1]:.2f}], "
          f"Z[{start}:{end}] = [{z_values[start]:.2f}, {z_values[end-1]:.2f}]")

# Создание графика
fig = go.Figure()

# Отрисовка профиля лопастей
if len(blade_x) > 0:
    for i, (start, end) in enumerate(blade_boundaries):
        fig.add_trace(go.Scatter(
            x=x_values[start:end],
            y=z_values[start:end],
            mode='lines+markers',
            name=f'Лопасть {i+1}',
            line=dict(color='blue' if i == 0 else 'red'),
            marker=dict(size=4),
            hovertemplate='X: %{x:.2f}<br>Z: %{y:.2f}'
        ))
else:
    print("Лопасти не найдены, отрисовка невозможна")

# Оформление графика
fig.update_layout(
    title='Профиль сечения лопастей (Z от X)',
    xaxis_title='X (мм)',
    yaxis_title='Z (мм)',
    hovermode='closest',
    showlegend=True,
    template='plotly_white'
)

# Отрисовка
fig.show()

Найдено лопастей: 2
Лопасть 1: X[39:627] = [-48.56, 4.09], Z[39:627] = [55.05, 42.62]
Лопасть 2: X[632:1011] = [5.05, 37.07], Z[632:1011] = [54.76, 42.76]
